In [1]:
#Mount your personal Google Drive to colab
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('drive')
os.chdir('MyDrive')
os.chdir("AI_for_IA") # move into the new subfolder => current working directory
os.chdir("MA4_LLMs")
os.getcwd() # check which is the current working directory

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/MyDrive/AI_for_IA/MA4_LLMs'

In [ ]:
# After navigating to your desired directory.
#This ensures that subsequent automatic and manual saves will occur in the folder you've specified.
%notebook "/content/drive/MyDrive/AI_for_IA/MA4_LLMs/MA4_AI_Agents_2.ipynb"

**1. Install libraries**


In [2]:
%%bash

pip -qqq install 'crewai[tools]'
pip -qqq install youtube-transcript-api
pip -qqq install yt_dlp

**2. Import libraries**


In [3]:
import os
from crewai import Agent
from google.colab import userdata
from crewai import Crew, Process
from crewai_tools import YoutubeChannelSearchTool
from crewai_tools import YoutubeVideoSearchTool
from crewai_tools import PDFSearchTool
from crewai import Task

In [4]:
OPENAI_API_KEY = userdata.get('OPEN_AI_KEY')
model_ID = userdata.get('GPT_MODEL')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["OPENAI_MODEL_NAME"] = model_ID

# **Agents and their tasks**

In [5]:
#The tool used by the topic researcher
youtube_tool = YoutubeChannelSearchTool(youtube_channel_handle='@techwithzoum')
#youtube_tool = YoutubeChannelSearchTool(youtube_channel_handle='@OpenDataScienceCon')
#pdftool = PDFSearchTool(pdf='MyDrive/AI_for_IA/MA4_LLMs/AI Agents Implementation in Python.pdf')


Inserting batches in chromadb: 100%|██████████| 1/1 [00:01<00:00,  1.45s/it]


In [6]:
#for showing the syntax:
youtube_tool
#pdftool

YoutubeChannelSearchTool(name='Search a Youtube Channels content', description="Tool Name: Search a Youtube Channels content\nTool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the Youtube Channels content', 'type': 'str'}}\nTool Description: A tool that can be used to semantic search a query the @techwithzoum Youtube Channels content.", args_schema=<class 'crewai_tools.tools.youtube_channel_search_tool.youtube_channel_search_tool.FixedYoutubeChannelSearchToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x7d88603015a0>, result_as_answer=False, summarize=False, adapter=EmbedchainAdapter(embedchain_app=<embedchain.app.App object at 0x7d887ec70190>, summarize=False), config=None)

In [7]:
print(Agent.__init__.__annotations__)  # Check parameter hints

{'data': 'Any', 'return': 'None'}


**1.Agents**

Definition of the class Agent from crewAI:
https://docs.crewai.com/concepts/agents

In [8]:
#Topic Researcher Agent
topic_researcher = Agent(
    role="Topic Researcher",
    goal="Search for relevant videos on the topic {topic} from the provided YouTube channel",
    backstory="Expert in finding and analyzing relevant content from YouTube channels, specializing in AI, Data Science, Machine Learning, and Generative AI topics.",
    #goal="Search for relevant content on the topic {topic} from the provided pdf document",
    #backstory="Expert in finding and analyzing relevant content from documents, specializing in AI, Data Science, Machine Learning, and Generative AI topics.",
    llm=model_ID,
    verbose=True,
    memory=True,
    tools=[youtube_tool],
    #tools=[pdftool],
    allow_delegation=True
)

#AI professor Agent
ai_professor_agent = Agent(
    role='AI Professor',
    goal='Provide a concise summary of the theoretical background required for understanding the transcription provided by the Topic Researcher',
    backstory = "Expert in AI and proficient in pedagocially explaining all relevant theoretical background required to understand the complex topics and the main results and conclusions reported by the author.",
    llm=model_ID,
    verbose = True,
    memory=True,
    tools=[],
    allow_delegation=False
)

#Technical Agent
technical_agent = Agent(
    role='Technical Implementation Expert',
    goal='Create a detailed explanation about all the technical requisites to successfully implement the procedure provided by the Topic Researcher',
    backstory = "Specialized in implementing LLM AI Agents in linux environments like google collab, know best practices and the most used libraries and platforms.",
    llm=model_ID,
    verbose = True,
    memory=True,
    tools=[],
    allow_delegation=False
)

# Report Writer Agent
report_writer_agent = Agent(
    role='Report Writer',
    goal='Write a comprehensive report from the transcription provided by the Topic Researcher, covering all necessary sections',
    backstory="Experienced in creating in-depth, well-structured summaries providing a general explanation of the approach/methodology used by the authors and the main results and conclusions they report.",
    llm=model_ID,
    verbose=True,
    memory=True,
    tools=[],
    allow_delegation=False
)

In [ ]:
#for debugging
print(vars(topic_researcher))

{'id': UUID('8660e257-30e5-4be5-b9e9-b67a4b851263'), 'formatting_errors': 0, 'role': 'Topic Researcher', 'goal': 'Search for relevant content on the topic {topic} from the provided pdf document', 'backstory': 'Expert in finding and analyzing relevant content from documents, specializing in AI, Data Science, Machine Learning, and Generative AI topics.', 'config': None, 'cache': True, 'verbose': True, 'max_rpm': None, 'allow_delegation': True, 'tools': [PDFSearchTool(name="Search a PDF's content", description='Tool Name: Search a PDF\'s content\nTool Arguments: {\'query\': {\'description\': "Mandatory query you want to use to search the PDF\'s content", \'type\': \'str\'}}\nTool Description: A tool that can be used to semantic search a query the MyDrive/AI_for_IA/MA4_LLMs/AI Agents Implementation in Python.pdf PDF\'s content.', args_schema=<class 'crewai_tools.tools.pdf_search_tool.pdf_search_tool.FixedPDFSearchToolSchema'>, description_updated=False, cache_function=<function BaseTool.<l

**2. Tasks**

In [9]:
# Tasks
research_task = Task(
    description="Identify and analyze aspects on the topic {topic} from the specified pdf.",
    expected_output="A complete word by word report on the most relevant content found on the topic {topic}.",
    #description="Identify and analyze the specified document.",
    #expected_output="A complete word by word report on the most relevant content found on the document.",
    agent=topic_researcher,
    tools=[youtube_tool],
    #tools=[pdftool]
)

ai_professor_task = Task(
    description="Create an academical article summarizing the key points from the transcription provided by the Topic Researcher, including potential applications .",
    expected_output="A markdown-formatted of the article in lecture format",
    agent=ai_professor_agent,
    output_file='ai_prof_lecture.md'
)


technical_task = Task(
    description="Create a technical report from the transcription provided by the Topic Researcher, including other platforms where the procedure could be implemented.",
    expected_output="A markdown-formatted of the technical report in a check list format",
    agent=technical_agent,
    output_file='technical_report.md'
)

report_task = Task(
    description=""" Write a comprehensive report based on the transcription provided by the Topic Researcher.
                    The article must include an introduction , step-by-step guides, and conclusion.
                    The overall content must be about 1200 words long.""",
    expected_output="A markdown-formatted of the report",
    agent=report_writer_agent,
    output_file='report.md'
)



**3.Run the Crew**

In [10]:
# Crew setup
my_crew = Crew(
    agents=[topic_researcher, ai_professor_agent, technical_agent, report_writer_agent],
    tasks=[research_task, ai_professor_task, technical_task, report_task],
    verbose=True,
    process=Process.sequential,
    memory=True,
    cache=True,
    max_rpm=100,
    share_crew=True
)


In [11]:
# Execute the crew's tasks
topic_of_interest = 'How to Build Anything With AI Agents - With Code'
#topic_of_interest = 'Orchestrating'
result = my_crew.kickoff(inputs={'topic': topic_of_interest})

print("-----------------------------")
print(result)

# Agent: Topic Researcher
## Task: Identify and analyze aspects on the topic How to Build Anything With AI Agents - With Code from the specified pdf.


# Agent: Topic Researcher
## Thought: I need to search for relevant videos on the topic "How to Build Anything With AI Agents - With Code" from the specified YouTube channel to gather detailed content on the topic.
## Using tool: Search a Youtube Channels content
## Tool Input: 
"{\"search_query\": \"How to Build Anything With AI Agents - With Code\"}"
## Tool Output: 
Relevant Content:
Complete video tutorial How to Build Anything With AI Agents - With CodeHow to Build Anything With AI Agents - With Code 12/16/24, 2:06 AM AI Agents — From Concepts to Practical Implementation in Python | by Zoumana Keita | Towards Data Science https:/medium.com/towards-data-science/ai-agents-from-concepts-to-practical-implementation-in-python-fb26789b1560 4/23

Lists Coding & Development 11 stories · 935 saves ChatGPT 21 stories · 910 saves AI Regulatio